# **Setup**

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from matplotlib import pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import metrics

#from sklearn.metrics import plot_roc_curve
from sklearn.metrics import RocCurveDisplay
from sklearn.decomposition import PCA
from matplotlib.pyplot import figure
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedShuffleSplit

import pandas as pd
import numpy as np

In [22]:
df = pd.read_csv('preprocessed_spam_ham_phishing.csv')

In [23]:
df.columns

Index(['hops', 'missing_subject', 'missing_to', 'missing_content-type',
       'missing_mime-version', 'missing_x-mailer',
       'missing_content-transfer-encoding', 'missing_x-mimeole',
       'missing_x-priority', 'missing_list-id', 'missing_lines',
       'missing_x-virus-scanned', 'missing_status', 'missing_content-length',
       'missing_precedence', 'missing_delivered-to',
       'missing_list-unsubscribe', 'missing_list-subscribe',
       'missing_list-post', 'missing_list-help', 'missing_x-msmail-priority',
       'missing_x-spam-status', 'missing_sender', 'missing_errors-to',
       'missing_x-beenthere', 'missing_list-archive', 'missing_reply-to',
       'missing_x-mailman-version', 'missing_x-miltered', 'missing_x-uuid',
       'missing_x-virus-status', 'missing_x-spam-level',
       'missing_x-spam-checker-version', 'missing_references',
       'missing_in-reply-to', 'missing_user-agent', 'missing_thread-index',
       'missing_cc', 'missing_received-spf', 'missing_x-orig

**Remove spam emails, only consider ham and phishing:**

In [24]:
df = df[df['label'] != 1]
print(df.shape)

(26508, 95)


In [25]:
df['label'].value_counts()

label
0    25220
2     1288
Name: count, dtype: int64

**Randomly Sample 1288 Ham emails to create a balanced dataset:**

In [26]:
df_ham = df[df['label'] == 0].sample(1288)
df_phish = df[df['label'] == 2]

In [27]:
df_phish = df_phish.assign(label=1)

In [28]:
df_new = df_ham._append(df_phish, ignore_index=True)
df_new

,hops,missing_subject,missing_to,missing_content-type,missing_mime-version,missing_x-mailer,missing_content-transfer-encoding,missing_x-mimeole,missing_x-priority,missing_list-id,...,domain_match_errors-to_reply-to,domain_match_sender_from,domain_match_references_reply-to,domain_match_references_in-reply-to,domain_match_references_to,domain_match_from_reply-to,domain_match_to_from,domain_match_to_message-id,domain_match_to_received,label
0,1,0,0,0,0,0,0,1,1,0,...,1,0,0,1,0,0,0,0,0,0
1,2,0,0,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,1,0,0,...,1,0,1,1,0,1,0,0,0,0
3,1,0,0,0,1,1,0,1,1,0,...,1,1,0,0,0,1,1,1,0,0
4,1,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2571,0,0,0,0,0,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2572,1,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,1,1,0,1
2573,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2574,1,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [29]:
df_new = df_new.sample(frac=1)
df = df_new.reset_index(drop=True)
df

,hops,missing_subject,missing_to,missing_content-type,missing_mime-version,missing_x-mailer,missing_content-transfer-encoding,missing_x-mimeole,missing_x-priority,missing_list-id,...,domain_match_errors-to_reply-to,domain_match_sender_from,domain_match_references_reply-to,domain_match_references_in-reply-to,domain_match_references_to,domain_match_from_reply-to,domain_match_to_from,domain_match_to_message-id,domain_match_to_received,label
0,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,1,0,1,1,0,...,1,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,1,0,1,1,1,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,1,1,1,1,0,...,1,1,1,1,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2571,0,0,0,0,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,1
2572,1,0,0,0,0,1,1,1,1,0,...,1,1,1,1,1,1,1,0,0,0
2573,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
2574,2,0,0,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


**Reduce feature set:**

The only features that are kept are domain matching features, as these should generalize across very different email datasets without issue.

In [30]:
feature_list = [
'domain_match_from_return-path',
'domain_match_message-id_from',
'domain_match_message-id_return-path',
'domain_match_to_from',
'domain_match_errors-to_from',
'domain_match_message-id_reply-to',
'domain_match_errors-to_message-id',
'domain_match_sender_from',
'domain_match_to_received',
'domain_match_errors-to_reply-to',
'domain_match_to_message-id',
'label']

feature_list = ['domain_val_message-id',
       'domain_match_message-id_from', 'domain_match_from_return-path',
       'domain_match_message-id_return-path', 'domain_match_message-id_sender',
       'domain_match_message-id_reply-to', 'domain_match_return-path_reply-to',
       'domain_match_reply-to_to', 'domain_match_to_in-reply-to',
       'domain_match_errors-to_message-id', 'domain_match_errors-to_from',
       'domain_match_errors-to_sender', 'domain_match_errors-to_reply-to',
       'domain_match_sender_from', 'domain_match_references_reply-to',
       'domain_match_references_in-reply-to', 'domain_match_references_to',
       'domain_match_from_reply-to', 'domain_match_to_from',
       'domain_match_to_message-id', 'domain_match_to_received', 'label']

df = df[feature_list]

In [31]:
df_Y = df['label']
df_X = df.drop('label', axis=1)

In [32]:
df_X.shape

(2576, 21)

In [33]:
features_list = df_X.columns

**Apply a standard scaler to the full data set:**

In [34]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(df_X)
df_X = scaler.transform(df_X)
df_X = pd.DataFrame(df_X, columns=features_list)

**Breaking the data into a test and training set (20% test, 80% train)**

In [66]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.20, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=42)


In [36]:
X_train.shape

(1648, 21)

In [37]:
X_test.shape

(516, 21)

# **Hyperparameter Tuning and Testing:**

**Random Forest:**

In [77]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("rf", RandomForestClassifier())
                ])

param_grid_list = {'rf__n_estimators': [100, 150],
                  'rf__criterion': ['entropy', 'gini'],
                  'rf__min_samples_split': [2, 3],
                  'rf__min_samples_leaf': [1, 2],
                  'rf__max_features': ['sqrt', 'log2']}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
rf_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)
pred_test = best_model.score(X_test, y_test)
print(pred_test)

# Test the best performing model on the test set
#predictions = best_model.predict(X_test)
predictions = best_model.score(X_val, y_val)
print(predictions)

# # Get the evaluation metrics
# print('Accuracy:', accuracy_score(y_test, predictions)*100)
# print('F1 Score:', f1_score(y_test, predictions)*100)
# print('Recall:', recall_score(y_test, predictions)*100)
# print('Precision:', precision_score(y_test, predictions)*100)
# print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
# print('Confusion Matrix:', confusion_matrix(y_test, predictions))

rf_df[rf_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 32 candidates, totalling 320 fits
[CV 1/10; 1/32] START rf__criterion=entropy, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100
[CV 2/10; 1/32] START rf__criterion=entropy, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100
[CV 3/10; 1/32] START rf__criterion=entropy, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100
[CV 4/10; 1/32] START rf__criterion=entropy, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100
[CV 5/10; 1/32] START rf__criterion=entropy, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100
[CV 6/10; 1/32] START rf__criterion=entropy, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100
[CV 7/10; 1/32] START rf__criterion=entropy, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_s

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rf__criterion,param_rf__max_features,param_rf__min_samples_leaf,param_rf__min_samples_split,param_rf__n_estimators,params,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.138624,0.034720,0.012660,0.011739,entropy,sqrt,1,2,100,"{'rf__criterion': 'entropy', 'rf__max_features...",...,0.985437,0.985437,0.970874,0.975728,0.956311,0.970874,0.980583,0.976699,0.009159,5
1,0.179523,0.055933,0.008063,0.003274,entropy,sqrt,1,2,150,"{'rf__criterion': 'entropy', 'rf__max_features...",...,0.985437,0.985437,0.970874,0.975728,0.956311,0.970874,0.980583,0.976699,0.009159,5
2,0.102943,0.020074,0.008205,0.004760,entropy,sqrt,1,3,100,"{'rf__criterion': 'entropy', 'rf__max_features...",...,0.985437,0.985437,0.970874,0.975728,0.956311,0.970874,0.980583,0.976699,0.009159,5
3,0.187048,0.048233,0.006590,0.002826,entropy,sqrt,1,3,150,"{'rf__criterion': 'entropy', 'rf__max_features...",...,0.985437,0.985437,0.970874,0.975728,0.956311,0.970874,0.980583,0.976699,0.009159,5
8,0.142725,0.038965,0.010624,0.008824,entropy,log2,1,2,100,"{'rf__criterion': 'entropy', 'rf__max_features...",...,0.985437,0.985437,0.970874,0.975728,0.956311,0.970874,0.980583,0.976699,0.009159,5


In [73]:
# # Validation
# pred = best_model.predict(X_val)
# print('Accuracy:', accuracy_score(y_val, pred)*100)

Accuracy: 97.57281553398059


**MLP:**

In [56]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("mlp", MLPClassifier())
                ])

param_grid_list = {'mlp__hidden_layer_sizes': [(20,), (20,20), (40,), (40,40)],
                   'mlp__activation': ['tanh', 'relu'],
                   'mlp__learning_rate': ['constant', 'adaptive'],
                   'mlp__solver': ['adam', 'sgd'],
                   'mlp__alpha': [0.0001, 0.001, 0.01]}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
mlp_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

mlp_df[mlp_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 96 candidates, totalling 960 fits
[CV 3/10; 1/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=adam
[CV 8/10; 1/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=adam
[CV 10/10; 1/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=adam
[CV 2/10; 1/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=adam
[CV 6/10; 1/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=adam
[CV 9/10; 1/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=adam
[CV 5/10; 1/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), ml

/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warning

[CV 5/10; 2/96] END mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=sgd;, score=0.964 total time=   0.3s
[CV 9/10; 2/96] END mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=sgd;, score=0.976 total time=   0.2s
[CV 2/10; 3/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=adaptive, mlp__solver=adam
[CV 3/10; 2/96] END mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=sgd;, score=0.964 total time=   0.3s
[CV 2/10; 2/96] END mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=constant, mlp__solver=sgd;, score=0.988 total time=   0.3s
[CV 3/10; 3/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=adaptive, mlp__solver=adam
[CV 4/10; 3/96] START mlp__activation=ta

/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warning

[CV 3/10; 4/96] END mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=adaptive, mlp__solver=sgd;, score=0.958 total time=   0.2s
[CV 2/10; 5/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20, 20), mlp__learning_rate=constant, mlp__solver=adam
[CV 2/10; 4/96] END mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=adaptive, mlp__solver=sgd;, score=0.982 total time=   0.3s
[CV 3/10; 5/96] START mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20, 20), mlp__learning_rate=constant, mlp__solver=adam
[CV 4/10; 4/96] END mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=adaptive, mlp__solver=sgd;, score=0.982 total time=   0.3s
[CV 6/10; 4/96] END mlp__activation=tanh, mlp__alpha=0.0001, mlp__hidden_layer_sizes=(20,), mlp__learning_rate=adaptive, mlp__solver=sgd;, score=0.952 total time=   0.2s
[CV 4/10; 5/96] START mlp__activat

...

/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 10/10; 94/96] END mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=constant, mlp__solver=sgd;, score=0.976 total time=   0.5s
[CV 1/10; 96/96] START mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=adaptive, mlp__solver=sgd
[CV 2/10; 95/96] END mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=adaptive, mlp__solver=adam;, score=0.994 total time=   0.3s
[CV 4/10; 95/96] END mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=adaptive, mlp__solver=adam;, score=0.982 total time=   0.3s
[CV 2/10; 96/96] START mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=adaptive, mlp__solver=sgd
[CV 7/10; 95/96] END mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=adaptive, mlp__solver=adam;, score=0.982 total time=   0.2s
[CV 3/10; 96/96] START mlp

/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warning

[CV 1/10; 96/96] END mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=adaptive, mlp__solver=sgd;, score=0.976 total time=   0.5s
[CV 3/10; 96/96] END mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=adaptive, mlp__solver=sgd;, score=0.964 total time=   0.5s
[CV 2/10; 96/96] END mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=adaptive, mlp__solver=sgd;, score=0.994 total time=   0.5s
[CV 4/10; 96/96] END mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=adaptive, mlp__solver=sgd;, score=0.982 total time=   0.5s
[CV 6/10; 96/96] END mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=adaptive, mlp__solver=sgd;, score=0.952 total time=   0.5s
[CV 7/10; 96/96] END mlp__activation=relu, mlp__alpha=0.01, mlp__hidden_layer_sizes=(40, 40), mlp__learning_rate=adaptive, mlp__solver=sgd;,

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_mlp__activation,param_mlp__alpha,param_mlp__hidden_layer_sizes,param_mlp__learning_rate,param_mlp__solver,params,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
62,0.265690,0.042839,0.001212,0.000601,relu,0.0001,"(40, 40)",adaptive,adam,"{'mlp__activation': 'relu', 'mlp__alpha': 0.00...",...,0.987879,0.957576,0.963636,0.981818,0.987879,0.987805,0.993902,0.979989,0.012127,1
72,0.197217,0.025439,0.001330,0.001315,relu,0.001,"(40,)",constant,adam,"{'mlp__activation': 'relu', 'mlp__alpha': 0.00...",...,0.987879,0.957576,0.963636,0.981818,0.987879,0.987805,0.993902,0.979989,0.012127,1
84,0.233514,0.042089,0.001001,0.000371,relu,0.01,"(20, 20)",constant,adam,"{'mlp__activation': 'relu', 'mlp__alpha': 0.01...",...,0.987879,0.957576,0.963636,0.981818,0.987879,0.987805,0.993902,0.979989,0.012127,1
88,0.226201,0.018720,0.001226,0.000781,relu,0.01,"(40,)",constant,adam,"{'mlp__activation': 'relu', 'mlp__alpha': 0.01...",...,0.987879,0.957576,0.963636,0.981818,0.987879,0.987805,0.993902,0.979989,0.012127,1
92,0.297469,0.047169,0.001517,0.001173,relu,0.01,"(40, 40)",constant,adam,"{'mlp__activation': 'relu', 'mlp__alpha': 0.01...",...,0.987879,0.957576,0.963636,0.981818,0.987879,0.987805,0.993902,0.979989,0.012127,1


**Logistic Regression:**

In [40]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("lr", LogisticRegression())
                ])

param_grid_list = {'lr__max_iter': [500],
                  'lr__solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
                  'lr__fit_intercept': [True, False],
                  'lr__tol': [0.0001, 0.001],
                  'lr__penalty': ['l1', 'l2', 'elasticnet'],
                  'lr__C': [0.1, 1, 10]}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
lr_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

lr_df[lr_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 144 candidates, totalling 1440 fits
[CV 1/10; 1/144] START lr__C=0.1, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=newton-cg, lr__tol=0.0001
[CV 2/10; 1/144] START lr__C=0.1, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=newton-cg, lr__tol=0.0001
[CV 1/10; 1/144] END lr__C=0.1, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=newton-cg, lr__tol=0.0001;, score=nan total time=   0.0s
[CV 3/10; 1/144] START lr__C=0.1, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=newton-cg, lr__tol=0.0001
[CV 3/10; 1/144] END lr__C=0.1, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=newton-cg, lr__tol=0.0001;, score=nan total time=   0.0s
[CV 2/10; 1/144] END lr__C=0.1, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=newton-cg, lr__tol=0.0001;, score=nan total time=   0.0s
[CV 4/10; 1/144] START lr__C=0.1, lr__fit_intercept=True, lr__max_iter=500

/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 9/10; 64/144] END lr__C=1, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l2, lr__solver=saga, lr__tol=0.001;, score=0.982 total time=   0.0s
[CV 10/10; 64/144] START lr__C=1, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l2, lr__solver=saga, lr__tol=0.001
[CV 4/10; 109/144] END lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l2, lr__solver=sag, lr__tol=0.0001;, score=0.982 total time=   0.1s
[CV 5/10; 109/144] START lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l2, lr__solver=sag, lr__tol=0.0001
[CV 10/10; 105/144] END lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l2, lr__solver=newton-cg, lr__tol=0.0001;, score=0.988 total time=   0.0s
[CV 1/10; 106/144] START lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l2, lr__solver=newton-cg, lr__tol=0.001
[CV 1/10; 106/144] END lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l2, lr__solver=newton-cg, lr__tol=0.001;, score=0.976 total time=  

/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/nat

[CV 3/10; 127/144] END lr__C=10, lr__fit_intercept=False, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.0001;, score=0.970 total time=   0.2s
[CV 7/10; 127/144] START lr__C=10, lr__fit_intercept=False, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.0001
[CV 7/10; 87/144] END lr__C=1, lr__fit_intercept=False, lr__max_iter=500, lr__penalty=l2, lr__solver=saga, lr__tol=0.0001;, score=0.982 total time=   0.1s
[CV 8/10; 87/144] START lr__C=1, lr__fit_intercept=False, lr__max_iter=500, lr__penalty=l2, lr__solver=saga, lr__tol=0.0001
[CV 3/10; 103/144] END lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.0001;, score=0.958 total time=   0.2s
[CV 4/10; 103/144] START lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.0001
[CV 4/10; 127/144] END lr__C=10, lr__fit_intercept=False, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.0001;, score=0.976 total time=   0.2s


...

[CV 10/10; 103/144] END lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.0001;, score=0.994 total time=   0.1s
[CV 1/10; 104/144] START lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.001
[CV 1/10; 104/144] END lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.001;, score=0.976 total time=   0.0s
[CV 2/10; 104/144] START lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.001
[CV 2/10; 104/144] END lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.001;, score=0.994 total time=   0.0s
[CV 3/10; 104/144] START lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.001
[CV 3/10; 104/144] END lr__C=10, lr__fit_intercept=True, lr__max_iter=500, lr__penalty=l1, lr__solver=saga, lr__tol=0.001;, score=0.958 total time=   0.0s
[CV 4/

/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
840 fits failed out of a total of 1440.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/pipeline.py", line 427, in

Accuracy: 97.48062015503875
F1 Score: 97.40518962075848
Recall: 98.78542510121457
Precision: 96.06299212598425
ROC AUC: 97.53397649112773
Confusion Matrix: [[259  10]
 [  3 244]]
CPU times: user 1.34 s, sys: 751 ms, total: 2.1 s
Wall time: 2.96 s


/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_lr__C,param_lr__fit_intercept,param_lr__max_iter,param_lr__penalty,param_lr__solver,param_lr__tol,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
102,0.164223,0.023305,0.000882,0.000156,10,True,500,l1,saga,0.0001,...,0.981818,0.957576,0.957576,0.981818,0.987879,0.987805,0.993902,0.977565,0.014087,1
103,0.053774,0.012734,0.000801,0.000116,10,True,500,l1,saga,0.001,...,0.981818,0.957576,0.957576,0.981818,0.987879,0.987805,0.993902,0.977565,0.014087,1
104,0.009498,0.004406,0.000941,0.000871,10,True,500,l2,newton-cg,0.0001,...,0.981818,0.957576,0.957576,0.981818,0.987879,0.987805,0.987805,0.976955,0.013486,3
105,0.007637,0.003794,0.000643,0.000225,10,True,500,l2,newton-cg,0.001,...,0.981818,0.957576,0.957576,0.981818,0.987879,0.987805,0.987805,0.976955,0.013486,3
106,0.011019,0.004900,0.001141,0.000799,10,True,500,l2,lbfgs,0.0001,...,0.981818,0.957576,0.957576,0.981818,0.987879,0.987805,0.987805,0.976955,0.013486,3


**SVM:**

In [41]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("svc", SVC())
                ])

param_grid_list = {'svc__C': [0.1, 1, 10],
                  'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                  'svc__degree': [3, 4, 5],
                  'svc__tol': [0.001, 0.0001, 0.01]}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
svm_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

svm_df[svm_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits
[CV 1/10; 1/108] START svc__C=0.1, svc__degree=3, svc__kernel=linear, svc__tol=0.001
[CV 2/10; 1/108] START svc__C=0.1, svc__degree=3, svc__kernel=linear, svc__tol=0.001
[CV 3/10; 1/108] START svc__C=0.1, svc__degree=3, svc__kernel=linear, svc__tol=0.001
[CV 2/10; 1/108] END svc__C=0.1, svc__degree=3, svc__kernel=linear, svc__tol=0.001;, score=0.988 total time=   0.0s
[CV 4/10; 1/108] START svc__C=0.1, svc__degree=3, svc__kernel=linear, svc__tol=0.001
[CV 1/10; 1/108] END svc__C=0.1, svc__degree=3, svc__kernel=linear, svc__tol=0.001;, score=0.952 total time=   0.0s
[CV 3/10; 1/108] END svc__C=0.1, svc__degree=3, svc__kernel=linear, svc__tol=0.001;, score=0.958 total time=   0.0s
[CV 5/10; 1/108] START svc__C=0.1, svc__degree=3, svc__kernel=linear, svc__tol=0.001
[CV 6/10; 1/108] START svc__C=0.1, svc__degree=3, svc__kernel=linear, svc__tol=0.001
[CV 7/10; 1/108] START svc__C=0.1, svc__degree=3, svc__kernel=linear, svc__to

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svc__C,param_svc__degree,param_svc__kernel,param_svc__tol,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
78,0.011119,0.004970,0.001931,0.000989,10,3,rbf,0.001,"{'svc__C': 10, 'svc__degree': 3, 'svc__kernel'...",0.975758,...,0.987879,0.957576,0.957576,0.981818,0.987879,0.987805,0.993902,0.979383,0.013046,1
79,0.010678,0.007220,0.002946,0.002551,10,3,rbf,0.0001,"{'svc__C': 10, 'svc__degree': 3, 'svc__kernel'...",0.975758,...,0.987879,0.957576,0.957576,0.981818,0.987879,0.987805,0.993902,0.979383,0.013046,1
80,0.009305,0.004827,0.001894,0.000723,10,3,rbf,0.01,"{'svc__C': 10, 'svc__degree': 3, 'svc__kernel'...",0.975758,...,0.987879,0.957576,0.957576,0.981818,0.987879,0.987805,0.993902,0.979383,0.013046,1
90,0.007910,0.002394,0.001670,0.000518,10,4,rbf,0.001,"{'svc__C': 10, 'svc__degree': 4, 'svc__kernel'...",0.975758,...,0.987879,0.957576,0.957576,0.981818,0.987879,0.987805,0.993902,0.979383,0.013046,1
91,0.007241,0.002631,0.001635,0.000727,10,4,rbf,0.0001,"{'svc__C': 10, 'svc__degree': 4, 'svc__kernel'...",0.975758,...,0.987879,0.957576,0.957576,0.981818,0.987879,0.987805,0.993902,0.979383,0.013046,1


**Decision Tree:**

In [42]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("dt", DecisionTreeClassifier())
                ])

param_grid_list = {'dt__criterion': ['entropy', 'gini'],
                  'dt__min_samples_split': [2, 3, 4],
                  'dt__min_samples_leaf': [1, 2, 3],
                  'dt__ccp_alpha': [0, 0.005, 0.01, 0.025, 0.05, 0.1]}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
dt_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

dt_df[dt_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits
[CV 1/10; 1/108] START dt__ccp_alpha=0, dt__criterion=entropy, dt__min_samples_leaf=1, dt__min_samples_split=2
[CV 2/10; 1/108] START dt__ccp_alpha=0, dt__criterion=entropy, dt__min_samples_leaf=1, dt__min_samples_split=2
[CV 3/10; 1/108] START dt__ccp_alpha=0, dt__criterion=entropy, dt__min_samples_leaf=1, dt__min_samples_split=2
[CV 1/10; 1/108] END dt__ccp_alpha=0, dt__criterion=entropy, dt__min_samples_leaf=1, dt__min_samples_split=2;, score=0.976 total time=   0.0s
[CV 4/10; 1/108] START dt__ccp_alpha=0, dt__criterion=entropy, dt__min_samples_leaf=1, dt__min_samples_split=2
[CV 2/10; 1/108] END dt__ccp_alpha=0, dt__criterion=entropy, dt__min_samples_leaf=1, dt__min_samples_split=2;, score=0.988 total time=   0.0s
[CV 3/10; 1/108] END dt__ccp_alpha=0, dt__criterion=entropy, dt__min_samples_leaf=1, dt__min_samples_split=2;, score=0.964 total time=   0.0s
[CV 5/10; 1/108] START dt__ccp_alpha=0, dt__criterion=entropy, dt

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_dt__ccp_alpha,param_dt__criterion,param_dt__min_samples_leaf,param_dt__min_samples_split,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002022,0.000152,0.000631,0.000053,0,entropy,1,2,"{'dt__ccp_alpha': 0, 'dt__criterion': 'entropy...",0.975758,...,0.981818,0.957576,0.957576,0.981818,0.987879,0.987805,0.993902,0.977565,0.012717,5
1,0.002068,0.000280,0.000623,0.000056,0,entropy,1,3,"{'dt__ccp_alpha': 0, 'dt__criterion': 'entropy...",0.975758,...,0.981818,0.957576,0.957576,0.981818,0.987879,0.987805,1.000000,0.978174,0.013601,1
2,0.001938,0.000269,0.000691,0.000245,0,entropy,1,4,"{'dt__ccp_alpha': 0, 'dt__criterion': 'entropy...",0.975758,...,0.981818,0.957576,0.957576,0.981818,0.987879,0.987805,1.000000,0.978174,0.013601,1
9,0.001690,0.000109,0.000561,0.000023,0,gini,1,2,"{'dt__ccp_alpha': 0, 'dt__criterion': 'gini', ...",0.975758,...,0.981818,0.957576,0.957576,0.981818,0.987879,0.987805,0.993902,0.977565,0.012717,5
10,0.001907,0.000540,0.000565,0.000025,0,gini,1,3,"{'dt__ccp_alpha': 0, 'dt__criterion': 'gini', ...",0.975758,...,0.981818,0.957576,0.957576,0.981818,0.987879,0.987805,1.000000,0.978174,0.013601,1


**Naive Bayes (Gaussian):**

In [43]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("gnb", GaussianNB())
                ])

param_grid_list = {'gnb__var_smoothing': [1E-9, 1E-10, 1E-8]}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
nb_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

nb_df[nb_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 3 candidates, totalling 30 fits
[CV 1/10; 1/3] START gnb__var_smoothing=1e-09...................................
[CV 2/10; 1/3] START gnb__var_smoothing=1e-09...................................
[CV 2/10; 1/3] END ....gnb__var_smoothing=1e-09;, score=0.879 total time=   0.0s
[CV 1/10; 1/3] END ....gnb__var_smoothing=1e-09;, score=0.879 total time=   0.0s
[CV 3/10; 1/3] START gnb__var_smoothing=1e-09...................................
[CV 4/10; 1/3] START gnb__var_smoothing=1e-09...................................
[CV 3/10; 1/3] END ....gnb__var_smoothing=1e-09;, score=0.879 total time=   0.0s
[CV 4/10; 1/3] END ....gnb__var_smoothing=1e-09;, score=0.873 total time=   0.0s
[CV 5/10; 1/3] START gnb__var_smoothing=1e-09...................................
[CV 6/10; 1/3] START gnb__var_smoothing=1e-09...................................
[CV 7/10; 1/3] START gnb__var_smoothing=1e-09...................................
[CV 5/10; 1/3] END ....gnb__var_smoothing=1e-09;

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gnb__var_smoothing,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001856,0.000208,0.000643,0.000072,0.0,{'gnb__var_smoothing': 1e-09},0.878788,0.878788,0.878788,0.872727,0.896970,0.842424,0.90303,0.915152,0.890244,0.890244,0.884715,0.018737,2
1,0.001715,0.000128,0.000600,0.000015,0.0,{'gnb__var_smoothing': 1e-10},0.878788,0.878788,0.878788,0.872727,0.890909,0.842424,0.90303,0.915152,0.890244,0.890244,0.884109,0.018426,3
2,0.001693,0.000076,0.000607,0.000027,0.0,{'gnb__var_smoothing': 1e-08},0.878788,0.878788,0.878788,0.872727,0.909091,0.842424,0.90303,0.915152,0.890244,0.890244,0.885928,0.019850,1


**AdaBoost:**

In [44]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("ab", AdaBoostClassifier())
                ])

param_grid_list = {'ab__n_estimators': [50, 100, 150, 200],
                  'ab__learning_rate': [0.95, 1, 1.05, 1.25, 1.5, 1.75, 2],
                  'ab__algorithm': ['SAMME', 'SAMME.R']}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
ab_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

ab_df[ab_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 56 candidates, totalling 560 fits
[CV 1/10; 1/56] START ab__algorithm=SAMME, ab__learning_rate=0.95, ab__n_estimators=50
[CV 2/10; 1/56] START ab__algorithm=SAMME, ab__learning_rate=0.95, ab__n_estimators=50
[CV 3/10; 1/56] START ab__algorithm=SAMME, ab__learning_rate=0.95, ab__n_estimators=50
[CV 4/10; 1/56] START ab__algorithm=SAMME, ab__learning_rate=0.95, ab__n_estimators=50
[CV 5/10; 1/56] START ab__algorithm=SAMME, ab__learning_rate=0.95, ab__n_estimators=50
[CV 6/10; 1/56] START ab__algorithm=SAMME, ab__learning_rate=0.95, ab__n_estimators=50
[CV 7/10; 1/56] START ab__algorithm=SAMME, ab__learning_rate=0.95, ab__n_estimators=50
[CV 8/10; 1/56] START ab__algorithm=SAMME, ab__learning_rate=0.95, ab__n_estimators=50
[CV 9/10; 1/56] START ab__algorithm=SAMME, ab__learning_rate=0.95, ab__n_estimators=50
[CV 10/10; 1/56] START ab__algorithm=SAMME, ab__learning_rate=0.95, ab__n_estimators=50
[CV 2/10; 1/56] END ab__algorithm=SAMME, ab__learning_rate=0.95, a

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_ab__algorithm,param_ab__learning_rate,param_ab__n_estimators,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
29,0.112053,0.009638,0.008374,0.004210,SAMME.R,0.95,100,"{'ab__algorithm': 'SAMME.R', 'ab__learning_rat...",0.969697,0.981818,...,0.975758,0.957576,0.957576,0.969697,0.981818,0.987805,0.981707,0.972103,0.010889,2
30,0.168133,0.020994,0.012109,0.004920,SAMME.R,0.95,150,"{'ab__algorithm': 'SAMME.R', 'ab__learning_rat...",0.969697,0.981818,...,0.975758,0.957576,0.957576,0.969697,0.981818,0.987805,0.981707,0.972103,0.010889,2
31,0.217333,0.010806,0.018788,0.007077,SAMME.R,0.95,200,"{'ab__algorithm': 'SAMME.R', 'ab__learning_rat...",0.969697,0.981818,...,0.975758,0.957576,0.957576,0.969697,0.981818,0.987805,0.981707,0.972103,0.010889,2
34,0.182332,0.019078,0.013439,0.004045,SAMME.R,1,150,"{'ab__algorithm': 'SAMME.R', 'ab__learning_rat...",0.969697,0.981818,...,0.975758,0.957576,0.957576,0.969697,0.981818,0.987805,0.981707,0.972103,0.010889,2
35,0.258009,0.021665,0.019929,0.005527,SAMME.R,1,200,"{'ab__algorithm': 'SAMME.R', 'ab__learning_rat...",0.969697,0.981818,...,0.975758,0.957576,0.957576,0.969697,0.981818,0.987805,0.981707,0.972103,0.010889,2


**GradientBoostingClassifier:**

In [45]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("gbc", GradientBoostingClassifier())
                ])

param_grid_list = {'gbc__max_features': ['auto', 'sqrt', 'log2'],
                   'gbc__learning_rate': [0.05, 0.1, 0.2, 0.25, 0.30, 0.35, 0.40, 0.5, 0.6, 0.7, 0.9],
                   'gbc__n_estimators': [100, 200]
                  }

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
gb_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

gb_df[gb_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 66 candidates, totalling 660 fits
[CV 1/10; 1/66] START gbc__learning_rate=0.05, gbc__max_features=auto, gbc__n_estimators=100
[CV 1/10; 1/66] END gbc__learning_rate=0.05, gbc__max_features=auto, gbc__n_estimators=100;, score=nan total time=   0.0s
[CV 2/10; 1/66] START gbc__learning_rate=0.05, gbc__max_features=auto, gbc__n_estimators=100
[CV 2/10; 1/66] END gbc__learning_rate=0.05, gbc__max_features=auto, gbc__n_estimators=100;, score=nan total time=   0.0s
[CV 3/10; 1/66] START gbc__learning_rate=0.05, gbc__max_features=auto, gbc__n_estimators=100
[CV 3/10; 1/66] END gbc__learning_rate=0.05, gbc__max_features=auto, gbc__n_estimators=100;, score=nan total time=   0.0s
[CV 4/10; 1/66] START gbc__learning_rate=0.05, gbc__max_features=auto, gbc__n_estimators=100
[CV 4/10; 1/66] END gbc__learning_rate=0.05, gbc__max_features=auto, gbc__n_estimators=100;, score=nan total time=   0.0s
[CV 5/10; 1/66] START gbc__learning_rate=0.05, gbc__max_features=auto, gbc__n

/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
220 fits failed out of a total of 660.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
73 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/nataliechow/Library/Python/3.9/lib/python/site-packages/sklearn/pipeline.py", line 427, in f

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gbc__learning_rate,param_gbc__max_features,param_gbc__n_estimators,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
15,0.106728,0.011502,0.001392,0.000480,0.2,sqrt,200,"{'gbc__learning_rate': 0.2, 'gbc__max_features...",0.969697,0.993939,...,0.987879,0.957576,0.963636,0.981818,0.987879,0.987805,0.993902,0.979383,0.01247,1
23,0.101734,0.010497,0.001419,0.000806,0.25,log2,200,"{'gbc__learning_rate': 0.25, 'gbc__max_feature...",0.969697,0.993939,...,0.987879,0.957576,0.963636,0.981818,0.987879,0.987805,0.993902,0.979383,0.01247,1
26,0.049848,0.006972,0.000984,0.000110,0.3,sqrt,100,"{'gbc__learning_rate': 0.3, 'gbc__max_features...",0.969697,0.993939,...,0.987879,0.957576,0.963636,0.981818,0.987879,0.987805,0.993902,0.979383,0.01247,1
28,0.053117,0.006429,0.001272,0.000666,0.3,log2,100,"{'gbc__learning_rate': 0.3, 'gbc__max_features...",0.969697,0.993939,...,0.987879,0.957576,0.963636,0.981818,0.987879,0.987805,0.993902,0.979383,0.01247,1
32,0.056378,0.012075,0.001321,0.000754,0.35,sqrt,100,"{'gbc__learning_rate': 0.35, 'gbc__max_feature...",0.969697,0.993939,...,0.987879,0.957576,0.963636,0.981818,0.987879,0.987805,0.993902,0.979383,0.01247,1


**KNN:**

In [46]:
%%time

pipe = Pipeline([("scale", StandardScaler()),
                ("knn", KNeighborsClassifier())
                ])

param_grid_list = {'knn__n_neighbors': [1, 10, 20],
                  'knn__weights': ['uniform', 'distance'],
                  'knn__p': [1, 2],
                  'knn__algorithm': ['auto'],
                  'knn__leaf_size': [15, 30, 45]}

grid = GridSearchCV(pipe, param_grid=param_grid_list, cv=10, n_jobs=-1, verbose=10)

# Find the best hyperparameters (using 10 fold CV with the hold out fold being the validation set)
grid.fit(X_train, y_train)

# Check the hyperparameter results
knn_df = pd.DataFrame(grid.cv_results_)
print(grid.best_params_, '\n')
print(grid.best_estimator_)

# Get the best performing model
best_model = grid.best_estimator_

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Test the best performing model on the test set
predictions = best_model.predict(X_test)

# Get the evaluation metrics
print('Accuracy:', accuracy_score(y_test, predictions)*100)
print('F1 Score:', f1_score(y_test, predictions)*100)
print('Recall:', recall_score(y_test, predictions)*100)
print('Precision:', precision_score(y_test, predictions)*100)
print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
print('Confusion Matrix:', confusion_matrix(y_test, predictions))

knn_df[knn_df['rank_test_score'] <= 5].head(5)

Fitting 10 folds for each of 36 candidates, totalling 360 fits
[CV 1/10; 1/36] START knn__algorithm=auto, knn__leaf_size=15, knn__n_neighbors=1, knn__p=1, knn__weights=uniform
[CV 2/10; 1/36] START knn__algorithm=auto, knn__leaf_size=15, knn__n_neighbors=1, knn__p=1, knn__weights=uniform
[CV 3/10; 1/36] START knn__algorithm=auto, knn__leaf_size=15, knn__n_neighbors=1, knn__p=1, knn__weights=uniform
[CV 4/10; 1/36] START knn__algorithm=auto, knn__leaf_size=15, knn__n_neighbors=1, knn__p=1, knn__weights=uniform
[CV 5/10; 1/36] START knn__algorithm=auto, knn__leaf_size=15, knn__n_neighbors=1, knn__p=1, knn__weights=uniform
[CV 6/10; 1/36] START knn__algorithm=auto, knn__leaf_size=15, knn__n_neighbors=1, knn__p=1, knn__weights=uniform
[CV 7/10; 1/36] START knn__algorithm=auto, knn__leaf_size=15, knn__n_neighbors=1, knn__p=1, knn__weights=uniform
[CV 8/10; 1/36] START knn__algorithm=auto, knn__leaf_size=15, knn__n_neighbors=1, knn__p=1, knn__weights=uniform
[CV 9/10; 1/36] START knn__algori

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_knn__algorithm,param_knn__leaf_size,param_knn__n_neighbors,param_knn__p,param_knn__weights,params,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
9,0.001541,0.000620,0.005087,0.001559,auto,15,20,1,distance,"{'knn__algorithm': 'auto', 'knn__leaf_size': 1...",...,0.975758,0.957576,0.957576,0.981818,0.987879,0.987805,0.987805,0.977561,0.012124,4
11,0.001566,0.000626,0.001653,0.000055,auto,15,20,2,distance,"{'knn__algorithm': 'auto', 'knn__leaf_size': 1...",...,0.981818,0.957576,0.957576,0.981818,0.987879,0.987805,0.987805,0.978167,0.012170,1
21,0.002214,0.001234,0.004843,0.001201,auto,30,20,1,distance,"{'knn__algorithm': 'auto', 'knn__leaf_size': 3...",...,0.975758,0.957576,0.957576,0.981818,0.987879,0.987805,0.987805,0.977561,0.012124,4
23,0.001841,0.001265,0.004313,0.004307,auto,30,20,2,distance,"{'knn__algorithm': 'auto', 'knn__leaf_size': 3...",...,0.981818,0.957576,0.957576,0.981818,0.987879,0.987805,0.987805,0.978167,0.012170,1
33,0.001662,0.000297,0.004935,0.000860,auto,45,20,1,distance,"{'knn__algorithm': 'auto', 'knn__leaf_size': 4...",...,0.975758,0.957576,0.957576,0.981818,0.987879,0.987805,0.987805,0.977561,0.012124,4


# **Stacked Testing:**

In [48]:
from sklearn.ensemble import StackingClassifier

'''
base_learners = [('rf', RandomForestClassifier(criterion='entropy', max_features='auto', min_samples_leaf=1, min_samples_split=3, n_estimators=100)), 
                ('mlp', MLPClassifier(max_iter=500, activation='relu', alpha=0.001, hidden_layer_sizes=(20,), learning_rate='adaptive', solver='adam')),
                ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=15, n_neighbors=20, p=1, weights='distance')), 
                ('svm', SVC(C=10, kernel='rbf', tol=0.001))]
'''

base_learners_set1 = [('rf', RandomForestClassifier(criterion='entropy', min_samples_leaf=1, min_samples_split=3, n_estimators=100)), 
                ('mlp', MLPClassifier(max_iter=500, activation='relu', alpha=0.001, hidden_layer_sizes=(20,), learning_rate='adaptive', solver='adam')),
                ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=15, n_neighbors=20, p=1, weights='distance'))]

base_learners_set2 = [('rf', RandomForestClassifier(criterion='entropy', min_samples_leaf=1, min_samples_split=3, n_estimators=100)), 
                ('mlp', MLPClassifier(max_iter=500, activation='relu', alpha=0.001, hidden_layer_sizes=(20,), learning_rate='adaptive', solver='adam')), 
                ('svm', SVC(C=10, kernel='rbf', tol=0.001))]

base_learners_set3 = [('rf', RandomForestClassifier(criterion='entropy', min_samples_leaf=1, min_samples_split=3, n_estimators=100)),
                ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=15, n_neighbors=20, p=1, weights='distance')), 
                ('svm', SVC(C=10, kernel='rbf', tol=0.001))]

base_learners_set4 = [('mlp', MLPClassifier(max_iter=500, activation='relu', alpha=0.001, hidden_layer_sizes=(20,), learning_rate='adaptive', solver='adam')),
                ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=15, n_neighbors=20, p=1, weights='distance')), 
                ('svm', SVC(C=10, kernel='rbf', tol=0.001))]

base_learners = []
base_learners.append(base_learners_set1)
base_learners.append(base_learners_set2)
base_learners.append(base_learners_set3)
base_learners.append(base_learners_set4)

for base_learner_group in base_learners:

    meta_learner = LogisticRegression()

    clf = StackingClassifier(estimators=base_learner_group, final_estimator=meta_learner)

    # Train the stacked model on the full training data
    clf.fit(X_train, y_train)

    predictions = clf.predict(X_test)

    # Get the evaluation metrics
    print('Accuracy:', accuracy_score(y_test, predictions)*100)
    print('F1 Score:', f1_score(y_test, predictions)*100)
    print('Recall:', recall_score(y_test, predictions)*100)
    print('Precision:', precision_score(y_test, predictions)*100)
    print('ROC AUC:', roc_auc_score(y_test, predictions)*100)
    print('Confusion Matrix:', confusion_matrix(y_test, predictions))
    print('-----------------------------------------\n')

Accuracy: 97.67441860465115
F1 Score: 97.6
Recall: 98.78542510121457
Precision: 96.44268774703558
ROC AUC: 97.71985009707569
Confusion Matrix: [[260   9]
 [  3 244]]
-----------------------------------------

Accuracy: 97.67441860465115
F1 Score: 97.6
Recall: 98.78542510121457
Precision: 96.44268774703558
ROC AUC: 97.71985009707569
Confusion Matrix: [[260   9]
 [  3 244]]
-----------------------------------------

Accuracy: 97.67441860465115
F1 Score: 97.6
Recall: 98.78542510121457
Precision: 96.44268774703558
ROC AUC: 97.71985009707569
Confusion Matrix: [[260   9]
 [  3 244]]
-----------------------------------------

Accuracy: 97.67441860465115
F1 Score: 97.6
Recall: 98.78542510121457
Precision: 96.44268774703558
ROC AUC: 97.71985009707569
Confusion Matrix: [[260   9]
 [  3 244]]
-----------------------------------------

